***
## Parte II
A segunda parte do trabalho visa verificar as características de organização física dos dados, registros,
blocos e arquivos nos nossos dois sistemas alvo. Para tanto, realizaremos várias cargas experimentais de
dados variando vários parâmetros de armazenamento e verificando a implicação destas variações no
tempo despendido e no espaço ocupado.

- _Alice Teles Lucena_
- _Igor de Souza Lima_
- _Nathália Rodrigues Machado dos Santos_

--------------------------------------------
### Tarefa 6 – Analisar e descrever os detalhes de armazenamento físico de dados no PostgreSQL.
Analisar e descrever os detalhes de armazenamento físico de dados no PostgreSQL. Construir uma tabela comparativa das principais características do sistema. Utilize, se necessário, diagramas, gráficos, etc.

__Entrega:__ Relatório com o resultado da análise e descrição apresentados no jupyter notebook

<span style="color:orange">__Resposta:__</span>

Componentes/Dados | Análise 
:------: | :------ 
Arquivo de Banco de Dados | Formato de armazenamento no nível de arquivos e diretórios. A configuração e os arquivos de dados usados ​​por um cluster de banco de dados são armazenados juntos no diretório de dados do cluster, comumente referido como PGDATA, e que costuma estar no diretório /var/lib/pgsql/data. <br><br>Vários clusters, gerenciados por diferentes instâncias do servidor, podem existir na mesma máquina. Além disso, o diretório PGDATA contém vários subdiretórios e arquivos de controle.
Mapa de Espaço Livre | Cada relação e índice, exceto para índices de hash, possui um mapa de espaço livre para acompanhar o espaço disponível na relação. Ele é armazenado junto com os dados da relação principal em uma bifurcação de relação separada. <br><br>O Mapa do Espaço Livre é organizado como uma árvore de páginas FSM (Free Space Map). As páginas FSM de nível inferior armazenam o espaço livre disponível em cada página heap (ou índice), usando um byte para representar cada uma dessas páginas. Ademais, os níveis superiores agregam informações dos níveis inferiores. <br><br>Dentro de cada página FSM existe uma árvore binária, armazenada em um array com um byte por nó. Cada nó folha representa uma página heap ou uma página FSM de nível inferior. Em cada nó não folha, o maior dos valores de seus filhos é armazenado. O valor máximo nos nós folha é, portanto, armazenado na raiz.
Mapa de Visibilidade | Cada relação de heap possui um Mapa de Visibilidade (VM) para rastrear quais páginas contêm apenas tuplas conhecidas por serem visíveis para todas as transações ativas. Além disso, ele também rastreia quais páginas contêm apenas tuplas congeladas. <br><br>Ele é armazenado junto com os dados da relação principal em uma bifurcação de relação separada, com o nome do número do filenode da relação, mais um sufixo _vm. <br><br>Uma observação importante é que os índices não têm VMs. O mapa de visibilidade armazena dois bits por página de heap. O primeiro bit, se definido, indica que a página está totalmente visível ou, em outras palavras, que a página não contém nenhuma tupla que precise ser limpa. Essas informações também podem ser usadas por varreduras somente de índice para responder a consultas usando apenas a tupla de índice. O segundo bit, se definido, significa que todas as tuplas na página foram congeladas. Isso significa que mesmo um vácuo antirretorno não precisa revisitar opágina. <br><br>O mapa é conservador no sentido de garantir que, sempre que um bit for definido, saibamos que a condição é verdadeira, mas se um bit não for definido, ele pode ou não ser verdadeiro. Os bits do mapa de visibilidade são definidos apenas pelo vácuo, mas são limpos por quaisquer operações de modificação de dados em uma página.
Tabelas (Relações) | As relações são armazenados em um arquivo separado. <br><br>Para relações comuns, esses arquivos são nomeados de acordo com o número do filenode da relação. Mas para relações temporárias, o nome do arquivo está no formato tBBB_FFF, onde BBB é o ID do servidor que criou o arquivo e FFF é o número do nó do arquivo.Em ambos os casos, cada tabela possui um mapa de espaço livre, que armazena informações sobre o espaço livre disponível na relação. <br><br>As relações também têm um mapa de visibilidade, para rastrear quais páginas não possuem tuplas mortas. <br><br>Quando uma tabela excede 1 GB, ela é dividida em segmentos de tamanho de gigabyte. Dessa forma, evita-se problemas em plataformas que possuem limitações de tamanho de arquivo. (1 GB é o tamanho do segmento padrão. O tamanho do segmento pode ser ajustado usando a opção de configuração --with-segsize ao construir o PostgreSQL.) 
Índices | Os índices, assim como as relações, também são armazenados em um arquivo separado. <br><br>Para índices comuns, esses arquivos são nomeados de acordo com o número do filenode do índice. Mas para índices temporários, o nome do arquivo está no formato tBBB_FFF, onde BBB é o ID do servidor que criou o arquivo e FFF é o número do nó do arquivo. Em ambos os casos, cada índice possui um mapa de espaço livre, que armazena informações sobre o espaço livre disponível. <br><br>Quando um índice excede 1 GB, ele é dividido em segmentos de tamanho de gigabyte. Dessa forma, evita-se problemas em plataformas que possuem limitações de tamanho de arquivo. (1 GB é o tamanho do segmento padrão. O tamanho do segmento pode ser ajustado usando a opção de configuração --with-segsize ao construir o PostgreSQL.) 
Linha da Tabela (Relação) | Cada relação é armazenada como um array de páginas de tamanho fixo (geralmente 8 kB). Em uma relação, todas as páginas são logicamente equivalentes, portanto, um determinado item (linha) pode ser armazenado em qualquer página. <br><br>Todas as linhas da tabela são estruturadas da mesma maneira. Há um cabeçalho de tamanho fixo (ocupando 23 bytes na maioria das máquinas), seguido por um bitmap nulo opcional, um campo de ID de objeto opcional e os dados do usuário.
Página do Banco de Dados | Uma página pode conter 5 partes: <br>1. PageHeaderData - 24 bytes de comprimento; Contém informações gerais sobre a página, incluindo ponteiros de espaço livre. <br>2. ItemIdData - Matriz de identificadores de itens apontando para os itens reais; Cada entrada é um par (deslocamento, comprimento); 4 bytes por item. <br>3. Espaço livre - O espaço não alocado; Novos identificadores de itens são alocados no início desta área, novos itens no final. <br>4. Unid - Os próprios itens reais. <br>5. Espaço Especial - Dados específicos do método de acesso ao índice; Métodos diferentes armazenam dados diferentes; Vazio em relações comuns.
Catálogo | Coleção de tabelas e visões que contêm metadados sobre o banco de dados. Ele armazena informações sobre os objetos do banco de dados, como tabelas, colunas, índices, funções, operadores e etc. O catálogo é um componente essencial do sistema de gerenciamento de banco de dados (SGBD), pois fornece uma estrutura organizada para armazenar e acessar essas informações.

--------------------------------------------
### Tarefa 7 – Analisar e descrever os detalhes dos seguintes sistemas de arquivo disponíveis no Linux: Ext2, Ext3, ReiserFS e XFS.
Analisar e descrever os detalhes dos seguintes sistemas de arquivo disponíveis no Linux: Ext2, Ext3, ReiserFS e XFS. Construir uma tabela comparativa das principais características de cada um dos dois sistemas. Utilize, se necessário, diagramas, gráficos, etc.

__Entrega:__ Relatório com o resultado da análise e descrição apresentador no jupyter notebook

<span style="color:orange">__Resposta:__</span>

Sistema de Arquivo | Descrição | Características Principais 
:------: | :------ | :------
Ext2 | O Ext2 foi o primeiro sistema de arquivos usado pelo Linux e é considerado uma extensão do sistema de arquivos original do Unix (Ext). O Ext2 é conhecido por seu desempenho e simplicidade. <br><br>Ele divide o espaço em blocos e armazena informações sobre arquivos, como metadados (nome, tamanho, permissões) e dados (conteúdo do arquivo), em blocos de dados. No entanto, o Ext2 não possui journaling, o que significa que, em caso de falha do sistema, é necessário realizar uma verificação manual do sistema de arquivos, o que pode levar algum tempo. <br><br>O Ext2 também não suporta compressão de arquivos nem snapshots. | - Tamanho máximo de arquivo: 2 terabytes<br>- Tamanho máximo de sistema de arquivos: 32 terabytes<br>- Número máximo de arquivos: 10 milhões<br>- Não possui journaling<br>- Não suporta compressão de arquivos<br>- Não suporta snapshots
Ext3 | O Ext3 é uma extensão do Ext2 que adiciona recursos de journaling, que são registros de transações que garantem a integridade dos dados em caso de falha do sistema. Isso torna o Ext3 mais confiável em comparação com o Ext2, pois, quando ocorre uma interrupção no sistema, o journaling permite que o sistema de arquivos seja restaurado para um estado consistente mais rapidamente. <br><br>O Ext3 possui as mesmas limitações de tamanho de arquivo e sistema de arquivos do Ext2, mas adiciona suporte para journaling e compressão de arquivos. No entanto, o Ext3 ainda não suporta snapshots. | - Tamanho máximo de arquivo: 2 terabytes<br>- Tamanho máximo de sistema de arquivos: 32 terabytes<br>- Número máximo de arquivos: 10 milhões<br>- Suporta journaling<br>- Suporta compressão de arquivos<br>- Não suporta snapshots
ReiserFS | O ReiserFS é um sistema de arquivos projetado para otimizar o desempenho de leitura e gravação de arquivos pequenos. <br><br>Ele usa uma estrutura de árvore balanceada (árvore B*) para organizar e armazenar os dados. A árvore balanceada melhora a eficiência na busca e recuperação de arquivos, especialmente em cargas de trabalho com muitos arquivos pequenos. <br><br>O ReiserFS suporta journaling para garantir a integridade dos dados e é capaz de restaurar rapidamente o sistema de arquivos após uma falha. No entanto, o ReiserFS não possui suporte nativo para compressão de arquivos, mas oferece suporte para snapshots, que são cópias instantâneas do sistema de arquivos em um determinado momento no tempo. | - Tamanho máximo de arquivo: 8 terabytes<br>- Tamanho máximo de sistema de arquivos: 16 terabytes<br>- Número máximo de arquivos: depende do tamanho do sistema de arquivos<br>- Suporta journaling<br>- Não suporta compressão de arquivos<br>- Suporta snapshots
XFS | O XFS é um sistema de arquivos de alto desempenho e escalonável originalmente desenvolvido pela Silicon Graphics para uso em sistemas Unix. O XFS suporta tamanhos de arquivo e sistema de arquivos muito grandes, de até 8 exabytes (64 bits) ou 16 terabytes (32 bits). <br><br>Ele usa uma estrutura de metadados avançada para gerenciar eficientemente o espaço em disco e permite um acesso rápido aos arquivos, mesmo em sistemas com muitos arquivos. <br><br>O XFS suporta journaling para garantir a consistência dos dados e permite a compressão de arquivos para economizar espaço em disco. Além disso, o XFS oferece suporte a snapshots, permitindo criar cópias instantâneas do sistema de arquivos em determinados momentos. | - Tamanho máximo de arquivo: 8 exabytes (64 bits) ou 16 terabytes (32 bits)<br>- Tamanho máximo de sistema de arquivos: 8 exabytes (64 bits) ou 16 terabytes (32 bits)<br>- Número máximo de arquivos: depende do tamanho do sistema de arquivos<br>- Suporta journaling<br>- Suporta compressão de arquivos<br>- Suporta snapshots


--------------------------------------------
### Tarefa 8 - Re-executar a carga de dados no PostgreSQL para vários tipos de arquivos e configurações distintas.
Os arquivos utilizados devem ter as seguintes características:
- 1 arquivo de registros longos (>100 Kb) e com poucos registros (10 mil)
- 1 arquivo de registros curtos (<4 Kb) e com muitos registros (1 milhão)
- 1 arquivo de registros longos (>100 Kb) e com muitos registros (1 milhão)
- 1 arquivo de registros variáveis (ex. muitos NULLS) e com muitos registros (1 milhão)

Para a geração destes arquivos, modificar os scripts configurados na __Tarefa 2__.
Para cada arquivo, devem ser usados os seguintes sistemas de arquivos: Ext2, Ext3 e XFS.
Para cada carga, medir o tempo necessário para geração dos arquivos e espaço ocupado no disco.

__Entrega:__ Tabelas comparativas para cada SGBD, arquivo e sistema de arquivos usados em
termos do tempo de execução e do espaço ocupado no disco.

<span style="color:orange">__Resposta:__</span>

--------------------------------------------
### Tarefa 9 – Analisar as tabelas da Tarefa 8 explicando os valores de tempo e espaço obtidos.
__Entrega:__ Relatório com o resultado da análise

<span style="color:orange">__Resposta:__</span>